In [1]:
import pandas as pd

from superfv import OutputLoader

In [2]:
def build_df(sim, cats, exclude_first=True):

    slc = slice(2, None) if exclude_first else slice(1, None)

    def get_total_attr(sim, cat, attr):
        times = [
            getattr(timer.data[cat], attr)
            for timer in sim.minisnapshots["stepper_timer"][slc]
        ]
        return sum(times)

    take_step = get_total_attr(sim, "take_step", "cum_time")

    data = []
    for cat in cats:
        data.append(
            dict(
                cat=cat,
                tot_time=get_total_attr(sim, cat, "cum_time"),
                per_time=get_total_attr(sim, cat, "cum_time") / take_step * 100,
                n_calls=get_total_attr(sim, cat, "n_calls"),
            )
        )

    return pd.DataFrame(data)

In [9]:
base_path = "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/"

# sim = OutputLoader(f"{base_path}p7/N_2048")
sim = OutputLoader(f"{base_path}ZS7/N_2048")
# sim = OutputLoader(f"{base_path}MM7/3revs/no_delta/rtol_1e-5/N_2048")

Successfully read simulation output from "/scratch/gpfs/jp7427/out/timing-of-2d-sine-wave/cupy/ZS7/N_2048"


In [10]:
cats = [
    "take_step",
    "compute_dt",
    "update_workspaces",
    "update_workspaces:shock_detector",
    "interpolate_faces",
    "integrate_fluxes",
    "integrate_fluxes:fallback",
    "integrate_fluxes:riemann_solver",
    "integrate_fluxes:quadrature",
    "compute_RHS",
    "zhang_shu_limiter",
    "MOOD_loop",
    "MOOD_loop:compute_fallback_fluxes",
    "MOOD_loop:detect_troubled_cells",
    "MOOD_loop:revise_fluxes",
    "slope_limiter:detect_smooth_extrema",
]


df = build_df(sim, cats)

In [11]:
df

,cat,tot_time,per_time,n_calls
0,take_step,4.951728,100.000000,9
1,compute_dt,0.012140,0.245166,9
2,update_workspaces,0.422244,8.527210,28
3,update_workspaces:shock_detector,0.054584,1.102330,28
4,interpolate_faces,2.131227,43.040056,54
5,integrate_fluxes,0.532386,10.751516,54
6,integrate_fluxes:fallback,0.328542,6.634887,54
7,integrate_fluxes:riemann_solver,0.148841,3.005836,54
8,integrate_fluxes:quadrature,0.036722,0.741596,54
9,compute_RHS,0.050207,1.013932,27
